# Censo - parte 3: filtros

Vamos a cargar el dataset del apartado anterior y obtener subconjuntos de población

In [ ]:
# Ensure we are connected
spark

## Carga de datos

In [ ]:
# Read the data
censo = spark.read.format('parquet').load('data/censo-2011.parquet')

In [ ]:
censo.rdd.getNumPartitions()

In [ ]:
censo = censo.repartition(8)

In [ ]:
censo.rdd.getNumPartitions()

## emigración

Personas que no han nacido en la provincia en la que viven

In [ ]:
emigrantes = censo.filter( censo.CPRO != censo.CPRON )

In [ ]:
cem = emigrantes.count()

In [ ]:
cem/censo.count()

Veamos el desglose por provincias

In [ ]:
emprov = emigrantes.groupBy('CPRO').count().toPandas()

In [ ]:
emprov = emprov.rename(columns={'count':'emig'})
emprov

Añadimos nombres de provincia

In [ ]:
import pandas as pd
prov = pd.read_csv('data/codes-prov.csv', encoding='utf-8', dtype={'CPRO':str} )

In [ ]:
emprov2 = emprov.merge(prov,on='CPRO').sort_values('emig', ascending=False)

In [ ]:
emprov2

## ratios

Esto eran valores absolutos. Calculemos ahora los ratios: porcentaje de habitantes de la provincia que no han nacido en ella

In [ ]:
poblacion = censo.groupBy('CPRO').count().toPandas().rename(columns={'count':'pob'})

In [ ]:
emprov3 = emprov2.merge(poblacion, on='CPRO')

In [ ]:
emprov3['ratio'] = emprov3.emig/emprov3.pob

In [ ]:
emprov3.sort_values('ratio', ascending=False)

## movimientos dentro de la provincia

In [ ]:
emig_int = censo.filter( (censo.CMUN != censo.CMUNN) & (censo.CPRON == censo.CPRO) )

In [ ]:
emig_int_prov = emig_int.groupBy('CPRO').count().withColumnRenamed('count','intra')

In [ ]:
emig_int_prov2 = emig_int_prov.toPandas().merge(poblacion, on='CPRO').merge(prov, on='CPRO')

In [ ]:
emig_int_prov2.sort_values('intra', ascending=False)

In [ ]:
emig_int_prov2['ratio'] = emig_int_prov2['intra']/emig_int_prov2['pob']

In [ ]:
emig_int_prov2.sort_values('ratio', ascending=False)